In [ ]:
import numpy as np
import pandas as pd

from ConvertCSV import DataReady
from FinRockDataFeeder import PdDataFeeder
from FinRockTradingEnv import TradingEnv
from FinRockRender import PygameRender
from FinRockScaler import ZScoreScaler
from FinRockReward import AccountValueChangeReward
from Indicators import BolingerBands, SMA, RSI, PSAR, MACD
from FinRockMetrics import AccountValue

In [ ]:
df = pd.read_csv("/home/zymantas/Desktop/Training_data/5minData/ATOMUSDT5min.csv")

DataReady(df)

df = df[:-1000] # leave 1000 for testing

In [ ]:
pd_data_feeder = PdDataFeeder(
    df = df,
    indicators = [
        BolingerBands(data=df, period=20, std=2),
    ]
)

In [ ]:
env = TradingEnv(
    data_feeder = pd_data_feeder,
    output_transformer = ZScoreScaler(),
    initial_balance = 1000.0,
    max_episode_steps = 288,
    window_size = 50,
    metrics = [
        AccountValue()
    ]
)
action_space = env.action_space
input_shape = env.observation_space.shape

env.save_config()

In [ ]:
pygameRender = PygameRender(frame_rate=288)

state, info = env.reset()
pygameRender.render(info)
rewards = 0.0
while True:
    # simulate model prediction, now use random action
    action = np.random.randint(0, action_space)

    state, reward, terminated, truncated, info = env.step(action)
    rewards += reward
    pygameRender.render(info)

    if terminated or truncated:
        print(info['states'][-1].account_value, rewards)
        rewards = 0.0
        state, info = env.reset()
        pygameRender.reset()